# Lasso regression with pipelines

In [19]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

### Load dataset

* Ames housing data
* 81 feature variables
* `SalePrice` is the target variable

In [2]:
FILE_PATH = '/workspaces/ML_pipeline/data/'
train = pd.read_csv(FILE_PATH + 'train.csv')
X_test = pd.read_csv(FILE_PATH + 'test.csv')

train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


### Prepare data

In [3]:
# Train-test split
X = train.drop('SalePrice', axis=1)
y = train['SalePrice']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=.3, random_state=1121218)

# All categorical columns
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# Columns that can be safely label encoded
good_label_cols = [col for col in object_cols if 
                   set(X_train[col]) == set(X_valid[col])]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))

# Drop categorical columns that will not be encoded
X_train = X_train.drop(bad_label_cols, axis=1)
X_valid = X_valid.drop(bad_label_cols, axis=1)

### Exploratory data analysis

In [4]:
X_train.describe().T.iloc[:10] # All numerical cols

,count,mean,std,min,25%,50%,75%,max
Id,1022.0,728.628180,417.491868,1.0,374.5,734.5,1082.00,1459.0
MSSubClass,1022.0,57.030333,42.861210,20.0,20.0,50.0,70.00,190.0
LotFrontage,838.0,70.190931,24.110495,21.0,60.0,70.0,80.00,313.0
LotArea,1022.0,10472.601761,8782.768055,1491.0,7560.0,9571.0,11742.50,164660.0
OverallQual,1022.0,6.071429,1.374094,1.0,5.0,6.0,7.00,10.0
OverallCond,1022.0,5.578278,1.101703,1.0,5.0,5.0,6.00,9.0
YearBuilt,1022.0,1971.221135,29.863975,1875.0,1954.0,1973.0,2000.00,2009.0
YearRemodAdd,1022.0,1984.813112,20.671520,1950.0,1966.0,1994.0,2003.75,2010.0
MasVnrArea,1015.0,101.768473,180.299391,0.0,0.0,0.0,160.00,1600.0
BsmtFinSF1,1022.0,441.294521,438.430750,0.0,0.0,381.0,707.50,2260.0


In [5]:
X_train.describe(include=object).T.iloc[:10] # All object cols

,count,unique,top,freq
MSZoning,1022,5,RL,809
Street,1022,2,Pave,1017
Alley,67,2,Grvl,37
LotShape,1022,4,Reg,654
LandContour,1022,4,Lvl,920
LandSlope,1022,3,Gtl,966
Neighborhood,1022,25,NAmes,156
BldgType,1022,5,1Fam,850
HouseStyle,1022,8,1Story,514
MasVnrType,400,3,BrkFace,306


#### Missing values

* 14 features have missing values
* A table of the features and count of missing values is shown below

In [6]:
has_missing = X_train.isnull().sum() > 0
X_train.isnull().sum()[has_missing]

LotFrontage     184
Alley           955
MasVnrType      622
MasVnrArea        7
BsmtQual         30
BsmtCond         30
BsmtExposure     31
BsmtFinType1     30
BsmtFinType2     31
FireplaceQu     480
GarageType       58
GarageYrBlt      58
GarageFinish     58
Fence           821
dtype: int64

#### Features

* 37 numerical features
* 24 categorical features

In [7]:
# Numerical features
numerical_features = X_train.select_dtypes(include='number').columns.tolist()
print(f'There are {len(numerical_features)} numerical features:')
print(numerical_features)

There are 37 numerical features:
['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']


In [8]:
# Categorical features
categorical_features = X_train.select_dtypes(exclude='number').columns.tolist()
print(f'There are {len(categorical_features)} categorical features:')
print(categorical_features)

There are 24 categorical features:
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'LandSlope', 'Neighborhood', 'BldgType', 'HouseStyle', 'MasVnrType', 'ExterQual', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'CentralAir', 'KitchenQual', 'FireplaceQu', 'GarageType', 'GarageFinish', 'PavedDrive', 'Fence', 'SaleCondition']


### Preprocess data

* Set up a pipeline to process all columns
* Numerical columns
  * `Simpleimputer`: fill missing values with the mean
  * `MinMaxScaler`: normalize values
* Categorical columns
  * `Simpleimputer`: fill missing values with the modal category
  * `OneHotEncoder`: one-hot encode categories
  * Use `handle_unknown='ignore'` in the OneHotEncoder to skip previously unseen labels
  * If the unknowns are not ignored, OneHotEncoder throws an error if there is a mismatch in labels between train and test

In [11]:
numeric_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('scale', MinMaxScaler())
])

categorical_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('one-hot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

#### Column transformation

* We have set up numerical and categorical pipelines
* Pipeline objects have a fit and transform methods (also fit_transform)
* We would need to call two separate fit_transform operations, specifying columns to act upon using `select_dtypes`
* Using the `ColumnTransformer` class to apply transformations to both numeric and categorical

In [13]:
full_processor = ColumnTransformer(transformers=[
    ('number', numeric_pipeline, numerical_features),
    ('category', categorical_pipeline, categorical_features)
])

full_processor.fit_transform(X_train)

array([[0.49451303, 0.58823529, 0.16846209, ..., 0.        , 1.        ,
        0.        ],
       [0.63443073, 0.        , 0.16846209, ..., 0.        , 0.        ,
        0.        ],
       [0.53017833, 0.        , 0.16780822, ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.97325103, 0.        , 0.16846209, ..., 0.        , 1.        ,
        0.        ],
       [0.98902606, 0.23529412, 0.21917808, ..., 0.        , 1.        ,
        0.        ],
       [0.50685871, 0.23529412, 0.15068493, ..., 0.        , 1.        ,
        0.        ]])

### Final pipeline with an estimator



In [18]:
lasso = Lasso(alpha=0.1)

lasso_pipeline = Pipeline(steps=[
    ('preprocess', full_processor),
    ('model', lasso)
])

_ = lasso_pipeline.fit(X_train, y_train)

# Make predictions
preds = lasso_pipeline.predict(X_valid)
mae = mean_absolute_error(y_valid, preds)
lasso_score = lasso_pipeline.score(X_valid, y_valid)

print(f"MAE = {mae:.3f}")
print(f"LASSO score = {lasso_score:.3f}")

MAE = 19587.856
LASSO score = 0.794


#### Use GridSearch on pipeline

* Lasso's main hyperparameter is `alpha` 
* It can take on any value between 0 to infinity
* We test a range of alpha values between 0 and 1 in 0.05 increments
* Ran an exhaustive Grid Search using a 10-fold cross-validation
* Best MAE score = 18015.357
* Best alpha = 0.95

In [20]:
param_dict = {'model__alpha': np.arange(0, 1, 0.05)}

search = GridSearchCV(lasso_pipeline, param_dict, 
                      cv=10, 
                      scoring='neg_mean_absolute_error')

_ = search.fit(X_train, y_train)

In [25]:
print(f"Best score: {abs(search.best_score_):.3f}")
print(f"Best alpha: {search.best_params_}")

Best score: 18015.357
Best alpha: {'model__alpha': 0.9500000000000001}


#### Reiterate on GridSearch

* The best `alpha` score is 0.95 which is on the edge of our initial interval [0, 1]
* We run the Grid search again over a wider range
* Best MAE score = 17051.596 (a large improvement over original score 18015.357)
* Best alpha = 181

In [26]:
param_dict = {'model__alpha': np.arange(1, 200, 5)}

search = GridSearchCV(lasso_pipeline, param_dict, 
                      cv=10, 
                      scoring='neg_mean_absolute_error')

_ = search.fit(X_train, y_train)

In [27]:
print(f"Best score: {abs(search.best_score_):.3f}")
print(f"Best alpha: {search.best_params_}")

Best score: 17051.596
Best alpha: {'model__alpha': 181}


#### Use the best parameters

* Define the pipeline using Lasso alpha value 181

In [29]:
lasso = Lasso(alpha=181)

final_lasso_pipe = Pipeline(steps=[
    ('preprocess', full_processor),
    ('model', lasso)
])

_ = final_lasso_pipe.fit(X_train, y_train)

# Make predictions
preds = final_lasso_pipe.predict(X_valid)
mae = mean_absolute_error(y_valid, preds)
lasso_score = lasso_pipeline.score(X_valid, y_valid)

print(f"MAE = {mae:.3f}")
print(f"LASSO score = {lasso_score:.3f}")

MAE = 18291.962
LASSO score = 0.794
